In [1]:
# 2017-10-05 LSTM

In [74]:
from __future__ import division
import sys, os, datetime
import pandas as pd
from collections import Counter
import functools
#import tqdm
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
import sklearn.metrics as skm
import sklearn.model_selection as skms
import pymorphy2

In [143]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Flatten
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
import keras.backend as K

In [4]:
#dftrain, dftest = pd.read_csv("../Data/train_task1_latest.csv"), pd.read_csv("../Data/test_task1_latest.csv")
dftrain, dftest = pd.read_csv("../Work/train_task1_lemma.csv"), pd.read_csv("../Work/test_task1_lemma.csv")

In [5]:
dftrain.shape, dftest.shape, dftrain.columns

((119398, 7),
 (74294, 6),
 Index([u'paragraph_id', u'question_id', u'paragraph', u'question', u'target',
        u'paragraphL', u'questionL'],
       dtype='object'))

In [242]:
num_rows  = len(dftrain) #10000 #00
train_x0  = dftrain.paragraphL[:num_rows].tolist()
train_x1  = dftrain.questionL[:num_rows].tolist()
train_y   = dftrain.target[:num_rows]
train_voc = train_x0+train_x1

In [243]:
import keras.preprocessing.text
tokens = keras.preprocessing.text.Tokenizer(num_words=1000)
xx = tokens.fit_on_texts(train_voc)
train_x0 = tokens.texts_to_matrix(train_x0) #, mode='tfidf')
train_x1 = tokens.texts_to_matrix(train_x1) #, mode='tfidf')
train_x0.shape, train_x1.shape

#tokens.__dict__

((119398, 1000), (119398, 1000))

In [244]:
max_features = train_x0.shape[1]
#model = Sequential()
#model.add(Embedding(max_features, 256))
#model.add(Input(shape=[max_feature], dtype='float32'))
#model.add(LSTM(256, dropout=0.2, recurrent_dropout=0.2,return_sequences=True,input_shape=[None,6]))
#model.add(Dense(50, activation='relu'))
#model.add(Dense(5, activation='sigmoid'))

K.clear_session()

i0 = Input(shape=[max_features]) #, dtype='float32')
x0 = Embedding(max_features, 16)(i0)
x0 = LSTM(16)(x0)

i1 = Input(shape=[max_features]) #, dtype='float32')
x1 = Embedding(max_features, 8)(i1)
x1 = LSTM(8)(x1)

xx = keras.layers.concatenate([x0, x1])

xx = Dense(16, activation='relu')(xx)
#xx = Dropout(0.25)(xx)
xx = Dense(64, activation='relu')(xx)
#xx = Dropout(0.25)(xx)
#xx = Dense(128, activation='relu')(xx)
#xx = Dropout(0.25)(xx)
##xx = Dense(256, activation='relu')(xx)
xx = Dropout(0.25)(xx)
#o = Dense(5, activation='sigmoid')(xx)
o = Dense(1, activation='sigmoid')(xx)

model = Model(inputs=[i0,i1], outputs=o)

model.summary()

model.compile(loss='binary_crossentropy', #'mae', #'mse', #'binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy','mse'])

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 1000)          0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, 1000)          0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 1000, 16)      16000       input_1[0][0]                    
____________________________________________________________________________________________________
embedding_2 (Embedding)          (None, 1000, 8)       8000        input_2[0][0]                    
___________________________________________________________________________________________

In [247]:
from keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint

In [249]:
batch_size   = 1024+1024 #+512
print (datetime.datetime.now())

reduce_lr1     = ReduceLROnPlateau(monitor='loss', factor=0.5,patience=2, min_lr=0.0000001,verbose=1)
early_stopping = EarlyStopping(monitor='loss', patience=10,min_delta=0.0001,verbose=1)

#filepath="../Temp/TempTempG5M5KFold2/V1-UNET-val_"+metric+"={val_"+metric+":.4f}-{"+metric+":.4f}---"+str(i)+".hdf5"
#checkpoint1 = ModelCheckpoint(filepath, monitor=metric,        verbose=1, save_best_only=True, mode='max')

hist = model.fit([train_x0,train_x1], train_y,
                  batch_size=batch_size,
                  #shuffle=True,
                  epochs=37+20,
                  initial_epoch=37,
                  callbacks=[reduce_lr1],
                  #validation_data=([x_train[-200:],z_train[-200:],a_train[-200:]], y_train[-200:]),
                  verbose=2)
print (datetime.datetime.now())

2017-10-06 14:05:52.750747
Epoch 38/57
313s - loss: 0.5023 - binary_accuracy: 0.7149 - mean_squared_error: 0.1687
Epoch 39/57
312s - loss: 0.4977 - binary_accuracy: 0.7150 - mean_squared_error: 0.1674
Epoch 40/57
312s - loss: 0.4972 - binary_accuracy: 0.7152 - mean_squared_error: 0.1672
Epoch 41/57
311s - loss: 0.4943 - binary_accuracy: 0.7190 - mean_squared_error: 0.1662
Epoch 42/57
312s - loss: 0.4930 - binary_accuracy: 0.7236 - mean_squared_error: 0.1657
Epoch 43/57
311s - loss: 0.4906 - binary_accuracy: 0.7225 - mean_squared_error: 0.1648
Epoch 44/57
312s - loss: 0.4891 - binary_accuracy: 0.7250 - mean_squared_error: 0.1642
Epoch 45/57
312s - loss: 0.4874 - binary_accuracy: 0.7267 - mean_squared_error: 0.1636
Epoch 46/57
313s - loss: 0.4866 - binary_accuracy: 0.7296 - mean_squared_error: 0.1632
Epoch 47/57
312s - loss: 0.4850 - binary_accuracy: 0.7298 - mean_squared_error: 0.1628
Epoch 48/57
311s - loss: 0.4865 - binary_accuracy: 0.7313 - mean_squared_error: 0.1633
Epoch 49/57
311s

In [250]:
#hist.__dict__

In [251]:
train_xp = model.predict([train_x0,train_x1],batch_size=batch_size)

In [256]:
skm.accuracy_score(train_y,train_xp>0.5), skm.roc_auc_score(train_y,train_xp)

(0.73461867032948625, 0.78626132406633753)